# Create general test case...

In [8]:
import numpy as np
from onnx import TensorProto, helper
import finn.core.onnx_exec as oxe
from finn.core.datatype import DataType

def setup_reference_model():
    channels = 128
    im_width = 256
    pe = 1
    global_in_dt = DataType.INT32
    
    
    in_shape = [1, NumChannels, 1, im_width]
    out_shape = [1, NumChannels, 1, im_width]

    
    # Create global input/outputs
    inp = helper.make_tensor_value_info("inp", TensorProto.FLOAT, in_shape)
    outp = helper.make_tensor_value_info("outp", TensorProto.FLOAT, out_shape)
    
    # Create intermediate nodes
    Thresholding_Batch_node0 = helper.make_node(
        "Thresholding_Batch",
        ["inp", "thresh0"],
        ["outp_thresholding0"],
        domain="finn.custom_op.fpgadataflow",
        backend="fpgadataflow",
        NumChannels=channels,
        PE=pe,
        numSteps=15,
        inputDataType=global_in_dt.name,
        weightDataType=DataType.INT32.name,  # will be set by MinimizeAccumulatorWidth
        outputDataType=DataType.UINT4.name,
        ActVal=0,
        mem_mode="const",
    )
    
    DuplicateStreams_Batch_node0 = helper.make_node(
        "DuplicateStreams_Batch",
        ["outp_thersholding0"],
        ["outp_duplicatestreamsbatch0,outp_duplicatestreamsbatch1"],
        domain="finn.custom_op.fpgadataflow",
        backend="fpgadataflow",
        NumChannels=channels,
        PE=pe,
        inputDataType=DataType.UINT4.name
    )
    
    FMPadding_Batch_node0 = helper.make_node(
        "FMPadding_Batch",
        ["outp_duplicatestreamsbatch0"],
        ["outp_fmpadding0"],
        domain="finn.custom_op.fpgadataflow",
        backend="fpgadataflow",
        ImgDim=[128,1],
        Padding[16,0,16,0],
        NumChannels=[256],
        SIMD=1,
        inputDataType=DataType.UINT4.name
    )
    
    ConvolutionInputGenerator_node0 = helper.make_node(
        "ConvolutionInputGenerator1D",
        ["outp_fmpadding0"],
        ["outp_convinpgen0"],
        
    )
    
    

In [12]:
from finn.core.datatype import DataType

a = DataType.INT32
print("{}\t{}".format(a, type(a)))
print("{}\t{}".format(a.name, type(a.name)))

DataType.INT32	<enum 'DataType'>
INT32	<class 'str'>


# Tested on QuartzNet

In [1]:
from finn.util.visualization import showInNetron
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition
from finn.transformation.fpgadataflow.floorplan import Floorplan
from finn.custom_op.registry import getCustomOp

#filename = "/workspace/results/ooc_vivado_log/best_effort_design/dataflow_partition0_c05e1m73/df_model_ooc.onnx"
filename = "/tmp/model_floorplan.onnx"

#showInNetron(filename)
model = ModelWrapper(filename)

model = model.transform(CreateDataflowPartition())

model.save("/tmp/model_post_sdfp.onnx")


In [2]:
from finn.util.visualization import showInNetron

showInNetron("/tmp/model_post_sdfp.onnx")

Serving '/tmp/model_post_sdfp.onnx' at http://0.0.0.0:8081


In [11]:
showInNetron("/workspace/results/temp/df_model_fifos.onnx")

Stopping http://0.0.0.0:8081
Serving '/workspace/results/temp/df_model_fifos.onnx' at http://0.0.0.0:8081


In [1]:
from finn.transformation.fpgadataflow.floorplan import Floorplan
from finn.core.modelwrapper import ModelWrapper
from finn.util.visualization import showInNetron
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition

model = ModelWrapper("/workspace/results/ooc_vivado_log/best_effort_design/dataflow_partition0_c05e1m73/df_model_ooc.onnx")

model = model.transform(Floorplan("/workspace/results/ooc_vivado_log/best_effort_design/manual_floorplan.json"))
model = model.transform(CreateDataflowPartition())

model.save("/tmp/model_new_floorplan.onnx")


/workspace/finn-base/src/finn/transformation/general.py:336: UserWarning: 
Unused HW configurations: IODMA_0, IODMA_1
  warnings.warn("\nUnused HW configurations: " + ", ".join(unused_configs))


DuplicateStreams_Batch_0	1
StreamingDataWidthConverter_Batch_0	1
0-------
1-------
2-------
Vector_Vector_Activate_Batch_0	3
StreamingDataWidthConverter_Batch_1	3
StreamingFCLayer_Batch_2	3
FMPadding_Batch_1	3
ConvolutionInputGenerator1D_1	3
Vector_Vector_Activate_Batch_1	3
StreamingDataWidthConverter_Batch_2	3
3-------
FMPadding_Batch_2	3
ConvolutionInputGenerator1D_2	3
Vector_Vector_Activate_Batch_2	3
StreamingDataWidthConverter_Batch_3	3
4-------
FMPadding_Batch_3	3
5-------
Vector_Vector_Activate_Batch_3	2
StreamingDataWidthConverter_Batch_4	2
StreamingFCLayer_Batch_5	2
FMPadding_Batch_4	2
6-------
Vector_Vector_Activate_Batch_4	1
StreamingDataWidthConverter_Batch_5	1
StreamingFCLayer_Batch_6	1
AddStreams_Batch_0	1
Thresholding_Batch_0	1
DuplicateStreams_Batch_1	1
StreamingDataWidthConverter_Batch_6	1
FMPadding_Batch_5	1
ConvolutionInputGenerator1D_5	1
Vector_Vector_Activate_Batch_5	1
StreamingDataWidthConverter_Batch_7	1
7-------
8-------
FMPadding_Batch_6	2
ConvolutionInputGenera

FMPadding_Batch_49	2
StreamingDataWidthConverter_Batch_77	2
ConvolutionInputGenerator1D_49	2
Vector_Vector_Activate_Batch_49	2
StreamingDataWidthConverter_Batch_78	2
62-------
AddStreams_Batch_9	2
Thresholding_Batch_9	2
63-------
StreamingDataWidthConverter_Batch_79	1
FMPadding_Batch_50	1
StreamingFCLayer_Batch_61	1
64-------
ConvolutionInputGenerator1D_50	0
Vector_Vector_Activate_Batch_50	0
StreamingDataWidthConverter_Batch_81	0
StreamingFCLayer_Batch_62	0
FMPadding_Batch_51	0
StreamingDataWidthConverter_Batch_82	0
ConvolutionInputGenerator1D_51	0
Vector_Vector_Activate_Batch_51	0
StreamingDataWidthConverter_Batch_83	0
65-------
FMPadding_Batch_52	0
StreamingDataWidthConverter_Batch_84	0
ConvolutionInputGenerator1D_52	0
Vector_Vector_Activate_Batch_52	0
StreamingDataWidthConverter_Batch_85	0
66-------
FMPadding_Batch_53	0
StreamingDataWidthConverter_Batch_86	0
ConvolutionInputGenerator1D_53	0
Vector_Vector_Activate_Batch_53	0
StreamingDataWidthConverter_Batch_87	0
67-------
FMPadding_

In [6]:
from finn.util.visualization import showInNetron

showInNetron("/tmp/model_new_floorplan.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/model_new_floorplan.onnx' at http://0.0.0.0:8081


In [9]:
showInNetron("/tmp/finn_dev_mirza/dataflow_partition2_8_yl7kiv/df_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition2_8_yl7kiv/df_model.onnx' at http://0.0.0.0:8081


# Tried with vitisbuild to find bug...

In [1]:
from finn.transformation.fpgadataflow.vitis_build import VitisBuild
from finn.core.modelwrapper import ModelWrapper

model = ModelWrapper("/workspace/results/ooc_vivado_log/best_effort_design/dataflow_partition0_c05e1m73/df_model_fifos.onnx")
model.set_tensor_layout(model.graph.node[0].input[0], ["N","H","W","C"])
model = model.transform(VitisBuild(1,2,3,floorplan_file="/workspace/results/ooc_vivado_log/best_effort_design/manual_floorplan_vivado_copy.json"))

model.save("/tmp/model_vitisbuild.onnx")

In [5]:
from finn.transformation.fpgadataflow.floorplan import Floorplan
from finn.core.modelwrapper import ModelWrapper
from finn.util.visualization import showInNetron
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition


model = ModelWrapper("/tmp/model_vitisbuild.onnx")

model = model.transform(Floorplan("/workspace/results/ooc_vivado_log/best_effort_design/manual_floorplan_vivado_copy.json"))
model = model.transform(CreateDataflowPartition(), cleanup=False)

model.save("/tmp/model_new_floorplan_vitisbuild.onnx")


0-------
1-------
DuplicateStreams_Batch_0	1
StreamingDataWidthConverter_Batch_0	1
2-------
3-------
4-------
Vector_Vector_Activate_Batch_0	3
StreamingDataWidthConverter_Batch_1	3
StreamingFCLayer_Batch_2	3
FMPadding_Batch_1	3
ConvolutionInputGenerator1D_1	3
Vector_Vector_Activate_Batch_1	3
StreamingDataWidthConverter_Batch_2	3
5-------
FMPadding_Batch_2	3
ConvolutionInputGenerator1D_2	3
Vector_Vector_Activate_Batch_2	3
StreamingDataWidthConverter_Batch_3	3
6-------
FMPadding_Batch_3	3
7-------
Vector_Vector_Activate_Batch_3	2
StreamingDataWidthConverter_Batch_4	2
StreamingFCLayer_Batch_5	2
FMPadding_Batch_4	2
8-------
Vector_Vector_Activate_Batch_4	1
StreamingDataWidthConverter_Batch_5	1
StreamingFCLayer_Batch_6	1
AddStreams_Batch_0	1
Thresholding_Batch_0	1
DuplicateStreams_Batch_1	1
StreamingDataWidthConverter_Batch_6	1
FMPadding_Batch_5	1
ConvolutionInputGenerator1D_5	1
Vector_Vector_Activate_Batch_5	1
StreamingDataWidthConverter_Batch_7	1
9-------
10-------
FMPadding_Batch_6	2
Con

FMPadding_Batch_49	2
StreamingDataWidthConverter_Batch_77	2
ConvolutionInputGenerator1D_49	2
Vector_Vector_Activate_Batch_49	2
StreamingDataWidthConverter_Batch_78	2
64-------
AddStreams_Batch_9	2
Thresholding_Batch_9	2
65-------
StreamingDataWidthConverter_Batch_79	1
FMPadding_Batch_50	1
StreamingFCLayer_Batch_61	1
66-------
ConvolutionInputGenerator1D_50	0
Vector_Vector_Activate_Batch_50	0
StreamingDataWidthConverter_Batch_81	0
StreamingFCLayer_Batch_62	0
FMPadding_Batch_51	0
StreamingDataWidthConverter_Batch_82	0
ConvolutionInputGenerator1D_51	0
Vector_Vector_Activate_Batch_51	0
StreamingDataWidthConverter_Batch_83	0
67-------
FMPadding_Batch_52	0
StreamingDataWidthConverter_Batch_84	0
ConvolutionInputGenerator1D_52	0
Vector_Vector_Activate_Batch_52	0
StreamingDataWidthConverter_Batch_85	0
68-------
FMPadding_Batch_53	0
StreamingDataWidthConverter_Batch_86	0
ConvolutionInputGenerator1D_53	0
Vector_Vector_Activate_Batch_53	0
StreamingDataWidthConverter_Batch_87	0
69-------
FMPadding_

In [2]:
from finn.util.visualization import showInNetron
showInNetron("/tmp/model_new_floorplan_vitisbuild.onnx")

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/model_new_floorplan_vitisbuild.onnx'

# Visualize graphs currently running experiments for

In [2]:
from finn.util.visualization import showInNetron
showInNetron("/workspace/results/temp/df_model_bitfile.onnx")

Serving '/workspace/results/temp/df_model_bitfile.onnx' at http://0.0.0.0:8081


In [11]:
from finn.transformation.fpgadataflow.annotate_resources import AnnotateResources
from finn.core.modelwrapper import ModelWrapper
from finn.util.visualization import showInNetron
from finn.custom_op.registry import getCustomOp

file="/workspace/results/ooc_vivado_log/best_effort_design/dataflow_partition0_c05e1m73/df_model_fifos.onnx"
model = ModelWrapper(file)
for n in model.graph.node:
    inst = getCustomOp(n)
    if n.op_type=="ConvolutionInputGenerator1D":
        inst.set_nodeattr("ram_style","block")
    if n.op_type=="StreamingFCLayer_Batch":
        inst.set_nodeattr("resType", "lut")
        inst.set_nodeattr("mem_mode", "decoupled")
        inst.set_nodeattr("ram_style", "distributed")
        inst.set_nodeattr("runtime_writeable_weights", 0)
model = model.transform(AnnotateResources('estimate'))
model.save("/tmp/model_annotated.onnx")
showInNetron("/tmp/model_annotated.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/model_annotated.onnx' at http://0.0.0.0:8081


# Extra visualizations

In [6]:
from finn.core.modelwrapper import ModelWrapper
from finn.util.basic import get_by_name
from finn.transformation.general import GiveUniqueNodeNames
from finn.transformation.infer_shapes import InferShapes

model = ModelWrapper("/tmp/model_new_floorplan_vitisbuild.onnx")
model = model.transform(GiveUniqueNodeNames())
model = model.transform(InferShapes())

for n in model.graph.node:
    path = get_by_name(n.attribute,"model").s.decode("utf-8")
    slr = get_by_name(n.attribute,"slr").i
    model_partition = ModelWrapper(path)
    
    count=0
    for n_p in model_partition.graph.node:
        if n_p.op_type=="StreamingFCLayer_Batch":
            count+=1
    print("{}\t{}\t{}".format(n.name, count, get_by_name(n.attribute,"slr").i))
    assert(count<2)
    
    

StreamingDataflowPartition_0	0	-1
StreamingDataflowPartition_1	1	1
StreamingDataflowPartition_2	0	2
StreamingDataflowPartition_3	1	1
StreamingDataflowPartition_4	1	3
StreamingDataflowPartition_5	1	3
StreamingDataflowPartition_6	1	3
StreamingDataflowPartition_7	1	2
StreamingDataflowPartition_8	1	1
StreamingDataflowPartition_9	1	1
StreamingDataflowPartition_10	1	2
StreamingDataflowPartition_11	1	2
StreamingDataflowPartition_12	1	2
StreamingDataflowPartition_13	1	2
StreamingDataflowPartition_14	1	1
StreamingDataflowPartition_15	1	1
StreamingDataflowPartition_16	1	1
StreamingDataflowPartition_17	1	1
StreamingDataflowPartition_18	1	1
StreamingDataflowPartition_19	1	1
StreamingDataflowPartition_20	1	1
StreamingDataflowPartition_21	1	1
StreamingDataflowPartition_22	1	1
StreamingDataflowPartition_23	1	1
StreamingDataflowPartition_24	1	1
StreamingDataflowPartition_25	1	1
StreamingDataflowPartition_26	1	1
StreamingDataflowPartition_27	1	2
StreamingDataflowPartition_28	1	3
StreamingDataflowPartit

In [20]:
print(len(model.graph.node))

99


In [25]:
from finn.core.modelwrapper import ModelWrapper
from finn.util.basic import get_by_name

model = ModelWrapper("/tmp/model_new_floorplan_vitisbuild.onnx")

for n in model.graph.node:
    path = get_by_name(n.attribute,"model").s.decode("utf-8")
    slr = get_by_name(n.attribute,"slr").i

    print("#SLR: {}".format(slr))
    print('showInNetron("'+str(path)+'")')

#SLR: -1
showInNetron("/tmp/finn_dev_mirza/dataflow_partition0_b6e7auno/df_model.onnx")
#SLR: 1
showInNetron("/tmp/finn_dev_mirza/dataflow_partition2_6w7w8p5f/df_model.onnx")
#SLR: 2
showInNetron("/tmp/finn_dev_mirza/dataflow_partition3_mo48bb09/df_model.onnx")
#SLR: 1
showInNetron("/tmp/finn_dev_mirza/dataflow_partition4_5117x44w/df_model.onnx")
#SLR: 3
showInNetron("/tmp/finn_dev_mirza/dataflow_partition5_tiq5o7e8/df_model.onnx")
#SLR: 3
showInNetron("/tmp/finn_dev_mirza/dataflow_partition6_qgpj_68d/df_model.onnx")
#SLR: 3
showInNetron("/tmp/finn_dev_mirza/dataflow_partition7_1d6zl626/df_model.onnx")
#SLR: 2
showInNetron("/tmp/finn_dev_mirza/dataflow_partition8_teiam7xn/df_model.onnx")
#SLR: 1
showInNetron("/tmp/finn_dev_mirza/dataflow_partition9_fc27xwba/df_model.onnx")
#SLR: 1
showInNetron("/tmp/finn_dev_mirza/dataflow_partition10_ti808u64/df_model.onnx")
#SLR: 2
showInNetron("/tmp/finn_dev_mirza/dataflow_partition11_yy56zok3/df_model.onnx")
#SLR: 2
showInNetron("/tmp/finn_dev_mirz

In [4]:
#SLR: 1
showInNetron("/tmp/finn_dev_mirza/dataflow_partition93_9gko65go/df_model.onnx")


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/finn_dev_mirza/dataflow_partition93_9gko65go/df_model.onnx'

In [9]:
#SLR: 2
showInNetron("/tmp/finn_dev_mirza/dataflow_partition2_6w7w8p5f/df_model.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition2_6w7w8p5f/df_model.onnx' at http://0.0.0.0:8081


In [18]:
#SLR: 1
showInNetron("/tmp/finn_dev_mirza/dataflow_partition2_q9s7b5n6/df_model.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition2_q9s7b5n6/df_model.onnx' at http://0.0.0.0:8081


In [19]:
#SLR: 3
showInNetron("/tmp/finn_dev_mirza/dataflow_partition3_mfgdlfay/df_model.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition3_mfgdlfay/df_model.onnx' at http://0.0.0.0:8081


In [20]:
#SLR: 3
showInNetron("/tmp/finn_dev_mirza/dataflow_partition4_p8psfzqu/df_model.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition4_p8psfzqu/df_model.onnx' at http://0.0.0.0:8081


In [21]:
#SLR: 3
showInNetron("/tmp/finn_dev_mirza/dataflow_partition5_tmnxmxsz/df_model.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition5_tmnxmxsz/df_model.onnx' at http://0.0.0.0:8081


In [22]:
#SLR: 2
showInNetron("/tmp/finn_dev_mirza/dataflow_partition6_sdbfrq4_/df_model.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition6_sdbfrq4_/df_model.onnx' at http://0.0.0.0:8081


In [23]:
#SLR: 1
showInNetron("/tmp/finn_dev_mirza/dataflow_partition7_25n1kfw_/df_model.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition7_25n1kfw_/df_model.onnx' at http://0.0.0.0:8081


In [24]:
#SLR: 1
showInNetron("/tmp/finn_dev_mirza/dataflow_partition8_ex6mk5n2/df_model.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition8_ex6mk5n2/df_model.onnx' at http://0.0.0.0:8081


In [25]:
#SLR: 2
showInNetron("/tmp/finn_dev_mirza/dataflow_partition9_fdlimwj0/df_model.onnx")


Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition9_fdlimwj0/df_model.onnx' at http://0.0.0.0:8081


In [26]:
#SLR: 2
showInNetron("/tmp/finn_dev_mirza/dataflow_partition10_l8wlru8z/df_model.onnx")

Stopping http://0.0.0.0:8081
Serving '/tmp/finn_dev_mirza/dataflow_partition10_l8wlru8z/df_model.onnx' at http://0.0.0.0:8081
